This function computes the gradient of the velocity-field $ \mathbf{v}(\mathbf{x}, t) $ using an auxiliary meshgrid.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (3,) | $ \mathbf{x} $ |
| X | array (Nx, Ny, Nz) | X-meshgrid|
| Y | array (Nx, Ny, Nz) | Y-meshgrid|
| Z | array (Nx, Ny, Nz) | Y-meshgrid|
| Interpolant | list (3,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $ <br /> Interpolant[3]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in z|
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| delta | list (3,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| aux_grid | float | ratio of auxiliary meshgrid to original meshgrid |
| grad_vel | array (3,3) | $ \nabla \mathbf{v}(\mathbf{x}, t) $ |

In [5]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

/Users/enalex/OneDrive/TBarrier/TBarrier/2D/Src


In [6]:
# Import numpy as np
import numpy as np

# Import function that computes velocity in the domain
from ipynb.fs.defs.velocity import velocity

In [3]:
def gradient_velocity(t, x, X, Y, Interpolant, periodic, bool_unsteady, dt_data, delta, aux_grid):
        
    rho_x = delta[0]
    rho_y = delta[1]
    rho_z = delta[2]
    
    grad_vel = np.zeros((3, 3))
    
    xL = x-np.array([1, 0, 0])*rho_x
    vLend = velocity(t, xL, X, Y, Z, Interpolant, periodic, bool_unsteady)
        
    xR = x+np.array([1, 0, 0])*rho_x
    vRend = velocity(t, xR, X, Y, Z, Interpolant, periodic, bool_unsteady)
        
    xD = x-np.array([0, 1, 0])*rho_y
    vDend = velocity(t, xD, X, Y, Z, Interpolant, periodic, bool_unsteady)
        
    xU = x+np.array([0, 1, 0])*rho_y
    vUend = velocity(t, xU, X, Y, Z, Interpolant, periodic, bool_unsteady)
    
    xB = x-np.array([0, 0, 1])*rho_z
    vBend = velocity(t, xD, X, Y, Z, Interpolant, periodic, bool_unsteady)
        
    xF = x+np.array([0, 0, 1])*rho_z
    vFend = velocity(t, xU, X, Y, Z, Interpolant, periodic, bool_unsteady)
    
    grad_vel[0, 0] = (vRend[0]-vLend[0])/(2*aux_grid*dx_data)
    grad_vel[1, 0] = (vUend[1]-vDend[1])/(2*aux_grid*dx_data)
    grad_vel[2, 0] = (vFend[2]-vBend[2])/(2*aux_grid*dx_data)
    
    grad_vel[0, 1] = (vUend[0]-vDend[0])/(2*aux_grid*dy_data)
    grad_vel[1, 0] = (vUend[1]-vDend[1])/(2*aux_grid*dy_data)
                
    return grad_vel